In [1]:
import aioredis

In [8]:
pool=await aioredis.create_redis_pool("redis://localhost:55000")

In [9]:
await pool.set("key", "value")

True

In [14]:
await pool.scan()

(0, [b'key'])

In [25]:
await pool.zrevrangebyscore("applications")

[b'application_1617631063375_35852',
 b'application_1617631063375_35856',
 b'application_1617631063375_35855',
 b'application_1617631063375_35854',
 b'application_1617631063375_35851',
 b'application_1617631063375_35853',
 b'application_1617631063375_35848',
 b'application_1617631063375_35845',
 b'application_1617631063375_35850',
 b'application_1617631063375_35849',
 b'application_1617631063375_35836',
 b'application_1617631063375_35847',
 b'application_1617631063375_35844',
 b'application_1617631063375_35842',
 b'application_1617631063375_35846',
 b'application_1617631063375_35841',
 b'application_1617631063375_35843',
 b'application_1617631063375_35839',
 b'application_1617631063375_35822',
 b'application_1617631063375_35840',
 b'application_1617631063375_35835',
 b'application_1617631063375_35838',
 b'application_1617631063375_35832',
 b'application_1617631063375_35834',
 b'application_1617631063375_35831',
 b'application_1617631063375_35829',
 b'application_1617631063375_35827',
 